<a href="https://colab.research.google.com/github/kaushik912/code_public/blob/main/blogging/genai/python/langchain-genai-tryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.utilities import SerpAPIWrapper


# -------------------- MEMORY (Conversation Context) --------------------
memory = ConversationBufferMemory(memory_key="chat_history")

# -------------------- TOOLS (Using Calculator as an Example) --------------------
def calculator_tool(query: str):
    """A simple calculator tool that evaluates mathematical expressions."""
    try:
        return eval(query)
    except Exception as e:
        return str(e)

calculator = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Useful for mathematical calculations. Input must be a valid expression."
)

# -------------------- PROMPT TEMPLATE --------------------
template = """You are a helpful AI assistant.
You will remember previous conversations and assist the user.

{chat_history}

User: {user_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"],
    template=template,
)

# -------------------- CHAIN (Processing the conversation) --------------------
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# -------------------- AGENT (Combining LLM, Memory, and Tools) --------------------
agent = initialize_agent(
    tools=[calculator],  # Add tools here
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True  # Enables detailed output of steps
)

# -------------------- TESTING --------------------
print("Starting AI Assistant...\n")

print(agent.run("What is 25 + 75?"))
print(agent.run("Now multiply the result by 2"))
print(agent.run("Tell me a fun fact about AI!"))
print(agent.run("What was the previous math answer?"))  # Memory in action!


<ipython-input-3-f47aa898885f>:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-3-f47aa898885f>:45: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
<ipython-input-3-f47aa898885f>:47: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
<ipython-input-3-f47aa898885f>:54: LangChainDeprecationWarning: LangChain agents will continue to be suppor

Starting AI Assistant...



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 25 + 75
Observation: 100
Thought:AI: The result of 25 + 75 is 100. How can I assist you further?

> Finished chain.
The result of 25 + 75 is 100. How can I assist you further?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: (100 * 2)
Observation: 200
Thought:Do I need to use a tool? No
AI: The result of multiplying 100 by 2 is 200. How can I assist you further?

> Finished chain.
The result of multiplying 100 by 2 is 200. How can I assist you further?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: One fun fact about AI is that it can be trained to play complex games like chess and Go at a superhuman level. It's fascinating how AI can learn and improve its skills over time! Is there anything else you'd like to know about AI?

> Finished chain.
One fun fact 